In [ ]:
import sqlite3

import pandas as pd
from pytrends.request import TrendReq

In [ ]:
con = sqlite3.connect("nba_api.sqlite")

In [ ]:
# retrieve list of nba teams from our database
teams = pd.read_sql_query("select distinct full_name from teams", con).full_name.tolist()

<div class="alert alert-block alert-info">
Use google trend's API to get the online interest (popularity) of teams over time

In [ ]:
# get google trend popularity of teams
pytrends = TrendReq(hl="en-US", retries=2, backoff_factor=0.1)

table_name = "popularity"
con.execute(f"drop table if exists {table_name}")
first = True
for team in teams:
    pytrends.build_payload([team], cat=0, timeframe="all")
    df = pytrends.interest_over_time().drop(["isPartial"], axis=1)
    df.columns = ["value"]
    df["team"] = team

    df.to_sql("popularity", con, if_exists="append")

    if first is True:
        first = False
        con.execute(f"CREATE UNIQUE INDEX uq_teams_id_season ON {table_name}(date, team)")